In [0]:
dbutils.widgets.text("table_name", "")
table_name = dbutils.widgets.get("table_name") or None

from pyspark.sql.utils import AnalysisException
from src.config.settings import LAYER_PATHS
from src.silver.setup.registry import SILVER_TABLES

def validate_path_exists(path: str):
    try:
        spark.read.format("delta").load(path)
    except AnalysisException:
        raise Exception(f"Path da tabela silver não encontrado: {path}")


def validate_delta_format(path: str):
    try:
        spark.read.format("delta").load(path)
    except Exception:
        raise Exception(f"Tabela não está em formato Delta: {path}")


def validate_table_registered(table_name: str):
    try:
        spark.table(table_name)
    except AnalysisException:
        raise Exception(f"Tabela silver não registrada no metastore: {table_name}")


def validate_not_empty_soft(df, table_name: str, warnings: list):
    if df.limit(1).count() == 0:
        warnings.append(f"[WARNING] Tabela silver {table_name} está vazia")


def validate_ingestion_column_soft(df, table_name: str, warnings: list):
    if "ingestion_ts" not in df.columns:
        warnings.append(
            f"[WARNING] Tabela silver {table_name} não possui coluna ingestion_ts"
        )


In [0]:
errors = []
warnings = []

for silver_name, cfg in SILVER_TABLES.items():

    if table_name and silver_name != table_name:
        continue

    print(f"\nValidando tabela silver: {silver_name}")

    silver_path = f"{LAYER_PATHS['silver']}/{cfg['silver_suffix']}"
    silver_table = cfg["silver_table"]

    try:
        # HARD checks
        validate_path_exists(silver_path)
        validate_delta_format(silver_path)
        validate_table_registered(silver_table)

        df = spark.read.format("delta").load(silver_path)

        # SOFT checks
        validate_not_empty_soft(df, silver_name, warnings)
        validate_ingestion_column_soft(df, silver_name, warnings)

        print(f"{silver_name} OK")

    except Exception as e:
        print(str(e))
        errors.append(str(e))


In [0]:
print("\n--- RESULTADO VALIDAÇÃO SILVER ---")

if warnings:
    print("\n WARNINGS:")
    for w in warnings:
        print(w)

if errors:
    raise Exception(
        "Validação da camada silver falhou:\n" + "\n".join(errors)
    )

print("\nValidação da camada silver concluída com sucesso")
